In [1]:
import openai
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from typing import List, Tuple
from requests.exceptions import HTTPError, ConnectionError


In [2]:

# Set up your OpenAI API key
openai.api_key = " " # actual API key

def generate_response(prompt: str) -> str:
    """
    Generate a response from GPT-3 based on the provided prompt.
    
    Parameters:
        prompt (str): The prompt to send to GPT-3.
    
    Returns:
        str: The response generated by GPT-3 or an empty string in case of an error.
    """
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",  # GPT-3 model used
            prompt=prompt,
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except (HTTPError, ConnectionError) as e:
        print(f"Error generating response: {e}")
        return ""


In [3]:

def check_coherence(response: str) -> bool:
    """
    Check the coherence of the response by looking for common logical connectors.
    
    Parameters:
        response (str): The response text from GPT-3.
    
    Returns:
        bool: True if the response contains logical connectors, False otherwise.
    """
    connectors = ["however", "therefore", "consequently", "moreover", "thus"]
    return any(connector in response.lower() for connector in connectors)


In [4]:

def check_consistency(prompt: str, response: str) -> float:
    """
    Check the consistency of the response with the prompt by comparing keywords.
    
    Parameters:
        prompt (str): The original prompt.
        response (str): The response text from GPT-3.
    
    Returns:
        float: A score representing the consistency of the response with the prompt.
    """
    prompt_keywords = [word.lower() for word in prompt.split()]
    response_keywords = [word.lower() for word in response.split()]
    common_keywords = set(prompt_keywords) & set(response_keywords)
    return len(common_keywords) / len(prompt_keywords)


In [5]:
def analyze_prompts(prompts: List[str]) -> Tuple[List[str], List[float], List[float]]:
    """
    Generate responses and analyze each prompt for coherence and consistency.
    
    Parameters:
        prompts (List[str]): A list of prompts to be analyzed.
    
    Returns:
        Tuple[List[str], List[float], List[float]]:
            - responses (List[str]): The responses generated for each prompt.
            - coherence_scores (List[float]): Coherence scores for each response.
            - consistency_scores (List[float]): Consistency scores for each response.
    """
    responses = [generate_response(prompt) for prompt in prompts]
    coherence_scores = [check_coherence(response) for response in responses]
    consistency_scores = [check_consistency(prompt, response) for prompt, response in zip(prompts, responses)]
    return responses, coherence_scores, consistency_scores

In [6]:
def visualize_scores(prompts: List[str], coherence_scores: List[float], consistency_scores: List[float]) -> None:
    """
    Visualize coherence and consistency scores using bar plots.
    
    Parameters:
        prompts (List[str]): The prompts analyzed.
        coherence_scores (List[float]): Coherence scores for each prompt.
        consistency_scores (List[float]): Consistency scores for each prompt.
    """
    try:
        # Create a DataFrame for visualization
        data = {
            'Prompt': prompts,
            'Coherence Score': coherence_scores,
            'Consistency Score': consistency_scores
        }
        df = pd.DataFrame(data)

        # Visualization of coherence scores
        plt.figure(figsize=(12, 6))
        sns.barplot(x='Prompt', y='Coherence Score', data=df)
        plt.xticks(rotation=45, ha='right')
        plt.title('Coherence Scores for Different Prompts')
        plt.tight_layout()  # Adjust layout to fit labels and titles
        plt.show()

        # Visualization of consistency scores
        plt.figure(figsize=(12, 6))
        sns.barplot(x='Prompt', y='Consistency Score', data=df)
        plt.xticks(rotation=45, ha='right')
        plt.title('Consistency Scores for Different Prompts')
        plt.tight_layout()  # Adjust layout to fit labels and titles
        plt.show()
    except Exception as e:
        print(f"Error during visualization: {e}")